In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import acquire
import explore

import sklearn.preprocessing
from sklearn.cluster import KMeans

seed = 21

In [2]:
df = acquire.get_wine()

In [3]:
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color_red
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


In [ ]:
df.color.value_counts()

In [ ]:
df.quality.value_counts()

### There are 2339 rows with outliers out of a total of 6497 rows, for now we will keep outliers

In [ ]:
train, X_train, y_train, X_val, y_val, X_test, y_test = explore.split_data(df,'quality')

In [ ]:
X_train_s,X_val_s,X_test_s = explore.scale_minmax(X_train,X_val,X_test)

In [ ]:
X_train_s.head()

In [ ]:
x = X_train[['alcohol','residual sugar']]
kmeans = KMeans(n_clusters=3)
kmeans.fit(x)
kmeans.predict(x)

In [ ]:
kmeans.predict(X_val[['alcohol','residual sugar']])

In [ ]:
explore.plot_inertia(x)

In [ ]:
X_train['cluster'] = kmeans.predict(x)

In [ ]:
sns.relplot(data=X_train,x='alcohol',y='pH',hue='cluster')

In [ ]:
sns.relplot(data=X_train,x='alcohol',y='pH',hue=X_train.cluster)

In [ ]:
sns.heatmap(train.corr())

In [ ]:
train.corr()['quality'].sort_values()

In [ ]:
explore.plot_clusters(X_train_s,'alcohol','density','volatile acidity')

In [ ]:
X_train_s,X_val_s,X_test= explore.cluster_Xsets(X_train_s,X_val_s,X_test,
                                        ['alcohol','density','volatile acidity','chlorides'])

In [ ]:
X_train_s.head()

In [ ]:
X_train_s.cluster.value_counts()

The proportion of acid to (alcohol + sugar) affects quality   
The proportion of acid to chlorides affects quality

In [ ]:
def acid_alcohol_proportion(df):
    df['acid_alc_sugar']=(df['fixed acidity']+df['volatile acidity']+df['citric acid']/3)/(df['alcohol']+df['residual sugar']/2)
    df['acid_chlor'] = (df['fixed acidity']+df['volatile acidity']+df['citric acid']/3)/df['chlorides']
    return df

In [ ]:
X_train_s = acid_alcohol_proportion(X_train_s)

In [ ]:
X_train_s